In [43]:
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64
import json

In [44]:
import sys
sys.path.append('..')
import classified

In [45]:
# for future api
fapi_passphrase = classified.FPASSPHRASE
fapi_key = classified.FKEY
fapi_secret = classified.FSECRET

In [46]:
api_passphrase = classified.SPASSPHRASE
api_key = classified.SKEY
api_secret = classified.SSECRET

session = Session()

In [47]:
def auth(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": '2',
        "Content-Type": "application/json"
    }
    return headers

## Spot Trade

In [48]:
class executeTrade:
    
    def spotOrder(self, side, symbol, price, size):
        url = 'https://api.kucoin.com/api/v1/orders'
        now = int(time.time() * 1000)
        data = {"clientOid": "01", "side": side, "symbol": symbol, "price": price,"size": size}
        data_json = json.dumps(data)
        str_to_sign = str(now) + 'POST' + '/api/v1/orders' + data_json
        signature = base64.b64encode(
            hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
        passphrase = base64.b64encode(
            hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
        headers = {
            "KC-API-SIGN": signature,
            "KC-API-TIMESTAMP": str(now),
            "KC-API-KEY": api_key,
            "KC-API-PASSPHRASE": passphrase,
            "KC-API-KEY-VERSION": '2',
            "Content-Type": "application/json" # specifying content type or using json=data in request
        }
        response = session.post(url, headers=headers, data=data_json)
        print(response.status_code)
        print(response.json())
        
    def futureOrder(self, side, symbol, leverage, price, size):
        url = 'https://api-futures.kucoin.com/api/v1/orders'
        now = int(time.time() * 1000)
        data = {"clientOid": "02", "side": side, "symbol": symbol, "leverage": leverage, "price": price,"size": size}
        data_json = json.dumps(data)
        str_to_sign = str(now) + 'POST' + '/api/v1/orders' + data_json
        signature = base64.b64encode(
            hmac.new(fapi_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
        passphrase = base64.b64encode(
            hmac.new(fapi_secret.encode('utf-8'), fapi_passphrase.encode('utf-8'), hashlib.sha256).digest())
        headers = {
            "KC-API-SIGN": signature,
            "KC-API-TIMESTAMP": str(now),
            "KC-API-KEY": fapi_key,
            "KC-API-PASSPHRASE": passphrase,
            "KC-API-KEY-VERSION": '2',
            "Content-Type": "application/json" # specifying content type or using json=data in request
        }
        response = session.post(url, headers=headers, data=data_json)
        print(response.status_code)
        print(response.json())

In [49]:
trade = executeTrade()

In [50]:
trade.spotOrder("buy", "ETH-USDT", "500", "0.01")

200
{'code': '200000', 'data': {'orderId': '62b3825d7f527f00016ce126'}}


In [52]:
trade.futureOrder("buy", "ETHUSDTM", "1", "500", "1")

200
{'code': '200000', 'data': {'orderId': '62b3827251f56e0001e8b465'}}
